<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Python Language: Decorators</h1>  

©2000-20 Chris Seddon 
</div>

## 1
Execute the following cell to activate styling for this tutorial

In [ ]:
from IPython.display import HTML
HTML(f"<style>{open('my.css').read()}</style>")

## 2
In this tutorial we will be investigation decorators - how to create and use them and what they bring to Python.  Decorators are used to modify or enhance what a function can do.  Essentially, decorators replace a function with an enhanced version.  The enhanced function usually calls the original function as part of its functionality, but also provides extra features.  

Think of a Christmas tree just bought from the shop.  What we want to do is decorate the tree with lights, tinsel and baubles.  We then have an enhanced or decorated tree.

Getting back to Python, let's take a look at how to create a simple decorator.  We will be creating a "trace" decorator.  This decorator will print a trace followed by calling the function the decorator replaces.  But first, let's create 3 functions that will end up being decorated:

In [ ]:
def square(x): 
    return x**2

def cube(x):
    return x**3
    
def quad(x):
    return x**4

## 3
A decorator is itself a function - its name can be any legal name.  What is important is that all decorators follow a definite pattern.  This pattern is in 3 parts:

* the decorator will take the original function as a parameter
* the decorator will define a local pointer to an enhanced function
* the decorator will return the local pointer

Thus a decorator's input is an 'old' function and its output is a 'new' function.  
The decorator will look like:

In [ ]:
def trace(fn):         # input is the original 'old' function
    def enhanced(n):   # define a local function that calls the old function
        print("this will be the trace")
        return fn(n)   # call the 'old' function
    return enhanced    # return the 'new' function

## 4
We will be changing the print statement above in the final version of the decorator, but it will suffice for now.  Note that the local function in the decorator has the same signature as our 3 sample functions above (square, cube and quad).  This is because the "enhance" function will replace one of the sample functions and therefore should have the same set of parameters.  

Let's see what happens when we call "trace" with the same functions as parameters:

In [ ]:
print( trace(square) )
print( trace(cube) )
print( trace(quad) )

## 5
As expected "trace" returns a pointer to the "enhance" function in each case.  Given that "trace" returns a function pointer taking one parameter, we can call it by passing a single number.  Each of the sample functions will behave similarly, so we will choose "cube" to show what is happening.

In [ ]:
f = trace(cube)    # get a pointer to the decorator's 'new' function
print( f(5) )      # call this function

## 6
The pointer "f" can be omitted in the above example if we combine the two lines.  This gives rise to a 2 sets of brackets.  Unusual, but logical, since we have 2 nested functions in the decorator.

In [ ]:
print( trace(cube)(5) )

## 7
Now let's improve the print statement giving rise to the trace:

In [ ]:
def trace(fn):
    def enhanced(n):
        print(f"calling cube with parameter {n}")
        return fn(n)
    return enhanced

print( trace(cube)(5) )

## 8
This will only work with "cube".  We can do better than that of course:

In [ ]:
def trace(fn):
    def enhanced(n):
        print(f"calling {fn} with parameter {n}")
        return fn(n)
    return enhanced

print( trace(square)(5) )
print( trace(cube)(5) )
print( trace(quad)(5) )

## 9
As a refinement we can use <pre>fn.\__name__</pre> which returns just the name of the function.

In [ ]:
def trace(fn):
    def enhanced(n):
        print(f"calling {fn.__name__}({n})")
        return fn(n)
    return enhanced

print( trace(square)(5) )
print( trace(cube)(5) )
print( trace(quad)(5) )

## 10
Using the double set of brackets as in the above example is how "Functional" Python works.  Some programmers are unfamiliar with this programming paradigm and feel uncomfortable calling functions in this way.  Because of this Python adds some "syntax sugar" to make this more palatable at the expense of clarity.  

The idea is to use the syntax <pre>@trace</pre>
in front of our sample functions to automatically call the "trace" function whenever a normal call is made.  Thus the following complete example is equivalent to the "Functional" code above.

In [ ]:
def trace(fn):
    def enhanced(n):
        print(f"calling {fn.__name__}({n})")
        return fn(n)
    return enhanced

@trace
def square(x): 
    return x**2

@trace
def cube(x):
    return x**3
    
@trace
def quad(x):
    return x**4

print( square(5) )
print( cube(5) )
print( quad(5) )

## 11
Using the @ notation simplifies the code and only a single set of brackets is required in the call.  While this is simpler, it masks the fact that "trace" is called each time.  The fact that "trace" is called is obvious in the "Functional" code.  When we say we have a decorator we normally mean we will use the @ notation rather than its "Functional" equivalent.

This completes our simple decorator.  

Decorators can be much more complicated than this, but all follow the basic design that you pass an 'old' function and get back a 'new' function.  

Several decorators are provided as part of the Python standard library and many others are defined in third party libraries.  Decorators come in all shapes and sizes, so we will only be able to touch the surface with our examples.

One of my favourite decorators is "memoizer".  This decorator is used to improve performance on recursive algorithms like generating Fibonacci numbers.  What we find with algorithms like this is that the recursion often recalculates the same Fibonacci number over and over again.

Here is a recursive Fibonacci function in which we print a message if n equals 5.  We can then see how many times F(5) is calculated as part of evaluating F(6) through F(13):

In [ ]:
def F(n):
    if n==5: print("5", end=",")
    return n if n < 2 else F(n-1) + F(n-2)

print( F(6) )
print( F(7) )
print( F(8) )
print( F(9) )
print( F(10) )
print( F(11) )
print( F(12) )
print( F(13) )

## 12
As you can see F(5) is calculated numerous times.  If you count the 5s you find that for each Fibonacci calculation we find F(5) is calculated:

* F(6): 1 time
* F(7): 2 times
* F(8): 3 times
* F(9): 5 times
* F(10): 8 times
* F(11): 13 times
* F(12): 21 times
* F(13): 34 times

I think I've seen those numbers before!

The idea behind "memoizer" is to avoid all these repeat calculations.  Instead, when each F(n) is calculated for the first time, it is placed in a cache.  Then subsequence calculations can read F(n) from the cache rather than recalculate it.  This speeds things up a great deal.

Let's create two versions of F(n), one using the memoizer and one not.  We can then compare timings.  But first, here is the "memoize" decorator:

In [ ]:
def memoize(f):
    cache = {}
    def inner(n):
        if n in cache:
            return cache[n]
        else:
            cache[n] = f(n)
            return cache[n]
    return inner

## 13
Now let's do the timings.  We are going to calculate F(8) one million times to get accurate timings.  In turns out that the non "memoize" function is far too slow for larger values, but 8 is a rasonable compromize: 

In [ ]:
@memoize
def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)

def Fib(n):
    return n if n < 2 else Fib(n-1) + Fib(n-2)

import timeit, sys
sys.setrecursionlimit(20000)  
print( f"with memoize: {timeit.timeit(stmt='fib(8)', setup='from __main__ import fib', number=1000000)}" )
print( f"using recursion: {timeit.timeit(stmt='Fib(8)', setup='from __main__ import Fib', number=1000000)}" )


## 14
The "memoize" function saves a huge amount of time even on small values of n.  The effect is much bigger as n increases.  

Next we will look at a built-in decorator, the so called "property" decorator.  We define a class called "Circle" that defines 2 decorator methods:<pre>area
perimeter</pre>

The idea behind this decorator is to make method calls look like attributes (we can leave off the brackets in the method call):

In [ ]:
import math

class Circle(object):
    def __init__(self, radius):
        self.radius = radius

    # computed properties (and hence read only)
    @property
    def area(self):
        return math.pi*self.radius**2
    
    @property
    def perimeter(self):
        return 2*math.pi*self.radius

## 15
Let's create circle objects and access the methods as though they were attributes:

In [ ]:
circle1 = Circle(10.0)
circle2 = Circle(20.0)
circle3 = Circle(30.0)

print(circle1.radius)
print(circle1.area)
print(circle1.perimeter)
print()

print(circle2.radius)
print(circle2.area)
print(circle2.perimeter)
print()

print(circle3.radius)
print(circle3.area)
print(circle3.perimeter)

## 16
In the above code looks as though the "Circle" objects have 3 attributes.  In fact, by looking at the code we know there is only one attribute.  The other "attributes" are really methods that fetch the calculated values and hence are read only.  It's as if these are computed attribues.

We can verify there is only one attribute by looking at the objects dictionaries:

In [ ]:
print(f"circle1's dict: {circle1.__dict__}")
print(f"circle2's dict: {circle2.__dict__}")
print(f"circle3's dict: {circle3.__dict__}")

## 17
A related decorator is the "property setter" decorator.  This builds on the "property" decorator and allows us to set values as well as read values.   

With the setter decorator we can attempt to modify these "imaginary" attributes.  Note however, there is only one real attribute <pre>theRadius</pre>
Therefore the setter decorator, despite appearances, must modify this real attribute.  We use floats to modify "circle1" and "circle2", but with "circle3" we attempt to change this attribute to a nonsensical value (a string); we arrange for the setter decorator to raise an exception in this case.

In [ ]:
class Circle(object):
    def __init__(self, radius):
        self.theRadius = radius

    @property
    def radius(self):
        return self.theRadius
    
    @radius.setter      # version 2.6
    def radius(self, value):
        if not isinstance(value, float):
            raise TypeError("Must be a float")
        self.theRadius = value

circle1 = Circle(10.0)
circle2 = Circle(20.0)
circle3 = Circle(30.0)

circle1.radius = 15.0
print(circle1.radius)

circle2.radius = 25.0
print(circle2.radius)

try:
    circle3.radius = "big"
except TypeError as e:
    print(e)

## 18
Amongst the many other examples we could look at, I'll bring your attention to a decorator with parameters.

Consider this "logging" decorator.  This is an enhanced version of the "trace" decorator we saw earlier.  The parameter is passed to the decorator via an outer function called "log".  The rest of the decorator is defined on familiar lines.  

In [ ]:
import logging

# define a parameterized decorator
def log(level):
    def logit(fn):
        def enhance(x):
            message = f"calling {fn.__name__}({x})"
            if(level == logging.DEBUG):    logging.debug(message)
            if(level == logging.INFO):     logging.info(message)
            if(level == logging.WARNING):  logging.warning(message)
            if(level == logging.ERROR):    logging.error(message)
            if(level == logging.CRITICAL): logging.critical(message)
            return fn(x)
        return enhance
    return logit
 
@log(logging.WARNING)
def square(x): 
    return x**2
@log(logging.DEBUG)
def cube(x):
    return x**3
@log(logging.CRITICAL)
def quad(x):
    return x**4

## 19
A little information about logging would be useful here.  The "logging" module defines 5 logging levels from the least important, "DEBUG", all the way up to "CRITICAL":

* DEBUG
* INFO
* WARNING
* ERROR
* CRITICAL

The calling program has to set the desired level of logging using <pre>logging.basicConfig(level=logging.CRITICAL)
</pre>
and then all calls of this level and above will be recorded.

Suppose we start with setting logging to the "DEBUG" level.  This means all levels of logging will be recorded:

In [ ]:
logging.basicConfig(level=logging.DEBUG)

print(square(4))
print(cube(5))
print(quad(10))

## 20
Now suppose we set the logging level to "WARNING".  This will mean "ERROR" and "CRITICAL" levels will also be recorded, but not "DEBUG" and "INFO".

Note, because of the way Jupyter notebooks works, we have to reload the "logging" module to reset this module.  
<pre>import importlib
importlib.reload(logging)</pre>
This would not be necessary when running Python normally. 

In [ ]:
import importlib
importlib.reload(logging)

logging.basicConfig(level=logging.WARNING)

print(square(4))
print(cube(5))
print(quad(10))

## 21
Finally, let's raise the level to "CRITICAL":

In [ ]:
import importlib
importlib.reload(logging)

logging.basicConfig(level=logging.CRITICAL)

print(square(4))
print(cube(5))
print(quad(10))